<img 
    style="position: absolute; 
           left: 60%; 
           top: 0; /* Added to ensure proper positioning */
           height: 900px; 
           width: 40%; /* Maintain the original width */
           object-fit: cover; /* Adjust if necessary */
           clip-path: inset(0px 50px 0px 50px round 10px);" 
    src= "_data\fig\buildingdamage.svg"
/>
</figure>


<h1 style="width: 60%; color: #EC6842; font-size: 55pt;">
    <Strong>
        IJsselsteinseweg 77
    </Strong>
</h1>

<h2 id="Background"><B>
    Rationale for the project<a class="anchor-link" href="#Background">&#182;</a>
    </B>
</h2>
<p style="text-align: justify; width: 60%; font-weight: normal;">
     This studies case study is the building IJsselsteinseweg 77 in Nieuwegein in the Netherlands, a deatched 2-layer unreinforced masonry structure from 1961 with 2 floors and a cellar which was eventually demolished circa 2015. We can evaluate the state of damage before its demolishing due to a Foundation research report that was commissioned by the owner to Sandra Lobbe Ispectie & Advies, a building consulting firm who carried out the building and foundation investigation on June 2019 and thereafter produced a report documenting the buildings measurements, characteristics, damage state and their own assessment on the necessary course of action or risk of the building integrity due to subsidence effects. The main assessment found that the current capacity provided by the building’s foundation was Insufficient and given the experienced deformations by the building and the damage state of the building, countermeasures should be implemented in a period of 1-5 years following the assessment date. 
</p>

## <strong> X | Imports</strong>

In [1]:
from bricks import house, LTSM, EM
from bricks.tools.plots import EM_plot, LTSM_plot, subsurface

import numpy as np

## <strong> 0 | Instantiate `HOUSE` object</strong>

### <strong> 0.1 | Structure dimensions and measurements</strong>

In [2]:
walls = {
    'Wall 1':{"x": np.array([0, 0, 0]), "y": np.array([0, 3.5, 7]), "z": np.array([0, -72, -152]), 'phi': np.array([1/200,1/200]), 'height': 5250, 'thickness': 27,'area': 390600},
    'Wall 2':{"x": np.array([0, 4.5, 8.9]), "y": np.array([7, 7, 7]), "z": np.array([-152, -163, -188]),  'phi': np.array([1/33,1/50]), 'height': 5250, 'thickness': 27,'area': 464625},
    'Wall 3':{"x": np.array([8.9, 8.9]), "y": np.array([3.6, 7]), "z": np.array([-149, -188]), 'phi': np.array([0,0]), 'height': 5250, 'thickness': 27,'area': 166950},
    'Wall 4':{"x": np.array([8.9, 10.8]), "y": np.array([3.6, 3.6]), "z": np.array([-149,-138]), 'phi': np.array([0,0]), 'height': 2850, 'thickness': 27,'area': 49020},
    'Wall 5':{"x": np.array([10.8, 10.8]), "y": np.array([0, 3.6]), "z": np.array([-104, -138]), 'phi': np.array([1/77,1/67]), 'height': 2850, 'thickness': 27,'area': 96720},
    'Wall 6':{"x": np.array([0, 5.2, 6.4, 8.9, 10.8]), "y": np.array([0, 0, 0, 0, 0]), "z": np.array([0, -42, -55, -75, -104]), 'phi': np.array([1/100,1/100]), 'height': 5000, 'thickness': 27, 'area': 517980},
}

damage =  { 'crack_1' : {'wall_id' : 'Wall 2','c_w': 4,'c_l': 890,},
            'crack_2' : {'wall_id' : 'Wall 1','c_w': 2,'c_l': 1670,},
            'crack_3' : {'wall_id' : 'Wall 2','c_w': 3,'c_l': 1868,},
            'crack_4' : {'wall_id' : 'Wall 7','c_w': 0.1,'c_l': 2000,},
            'crack_5' : {'wall_id' : 'Wall 2','c_w': 2,'c_l': 1760,},
            'crack_6' : {'wall_id' : 'Wall 2','c_w': 1,'c_l': 994,},
            'crack_7' : {'wall_id' : 'Wall 2','c_w': 0.1,'c_l': 500,},
            'crack_8' : {'wall_id' : 'Wall 2','c_w': 2,'c_l': 993,}}

ijsselsteinseweg = house(measurements = walls)
ijsselsteinseweg.state = damage

### <strong> 0.2 | Estimating the displacement surface & greenfield profile</strong>

In [3]:
ijsselsteinseweg.interpolate() # Interpolate subsidence shape
params = ijsselsteinseweg.soil['house'].values()
ijsselsteinseweg.fit_function(i_guess = 1, tolerance = 1e-2, step = 1) # Fit gaussian shapes to walls

app = subsurface(ijsselsteinseweg, *params)
app.run_server(debug=False)

From the above data we can observe how all curves increasingly progress towards the maximum displacement but do not resemble a gaussian shape which we assume to be the shape that subsidence patterns take. As a consequence we require to try to extrapolate outside the data to conitnuate the subisdence shape and we make use of an optimisation algorithm in order to find the values of the different variables in the function.   

## <strong> 1 | Assessing the damage of the building</strong>

### <strong> 1.1 | Assesment through Empirical Methods </strong>

In [4]:
ijsselsteinseweg.SRI() #Compute the SRI parameters
report = EM(ijsselsteinseweg.soil['sri'])

app = EM_plot(report)
app.run_server(debug=False)

### <strong> 1.2 | Assesment through the LTSM </strong>

Include further assumptions as conditions
2. TILT VERIFICATION??
3. Height in sagging zone NA assumed to be at t= H/2 where as for hogging we assume the NA to be at the lower edge therfore t=H 
2. IDEA: Define as function determine potential maximum strain values and calibrate E/G ratio

In [5]:
limit_line = -1
eg_rat =  11  #Reccomended 12.6 ## May vary # Calibrate
LTSM(ijsselsteinseweg, limit_line, eg_rat, methods = ['greenfield','measurements'])

app = LTSM_plot(ijsselsteinseweg)    
app.run_server(debug=True, dev_tools_ui=False)

### <strong> Y | Proces save data </strong>


In [20]:
dicts = [ijsselsteinseweg.soil['sri'],
         ijsselsteinseweg.assessment['ltsm']['greenfield']['results'],
         ijsselsteinseweg.assessment['ltsm']['measurements']['results']]
names = ['SRI', 'LTSM-GF','LTSM-MS', 'VARIABLES']
ijsselsteinseweg.process_dfs(dicts, names)

In [7]:
ijsselsteinseweg.dataframes['SRI']

,Smax,dSmax,Defrat,drat,omega,phi,beta
0,152,152,0.021714,4.000000,0.010285,0.022853,0.033139
1,188,36,0.004045,7.202247,0.001236,0.005682,0.006918
2,188,39,0.011471,0.000000,0.011470,0.011470,0.022940
3,149,11,0.005789,0.000000,0.005789,0.005789,0.011579
4,138,34,0.009444,0.000000,0.009444,0.009444,0.018888
5,104,104,0.009630,7.840909,0.003889,0.015262,0.019151


In [18]:
ijsselsteinseweg.dataframes['LTSM']

,e_tot,e_bt,e_dt,e_bh,e_bs,e_sh,e_ss,e_h,l_h,l_s,lh_s,lh_h,dl_s,dl_h
0,0.021362,0.021362,0.000000,0.018424,0.021362,0.087131,0.008570,0.000000,3947.199407,3052.800593,1.162972,0.751848,0.071490,0.017694
1,0.017139,0.017139,-0.001495,0.020178,0.019366,0.032732,0.006567,-0.003039,0.000000,8900.000000,3.390476,0.000000,0.031887,0.000000
2,0.027792,0.027792,0.000000,0.024159,0.027792,0.102586,0.011058,0.000000,0.000000,3400.000000,1.295238,0.000000,0.084676,0.000000
3,0.041364,0.041364,0.000000,0.036042,0.041364,0.148674,0.016417,0.000000,0.000000,1900.000000,1.333333,0.000000,0.122944,0.000000
4,0.032279,0.032279,0.000371,0.030224,0.031511,0.065800,0.011418,0.000767,0.000000,3600.000000,2.526316,0.000000,0.059003,0.000000
5,0.013452,0.013452,0.000000,0.012148,0.013452,0.036461,0.005150,0.000000,6218.698494,4581.301506,1.832521,1.243740,0.031024,0.007770


In [21]:
ijsselsteinseweg.dataframes['LTSM-MS']

,e_tot,e_bt,e_dt,e_bh,e_bs,e_sh,e_ss,e_h,gfl_sag_,gfl_hog_
0,0.000168,0.000168,0.000000,0.000168,0.000000,0.000691,0.000000,0.000000,[],"[{'l_h': 7000.0, 'lh_h': 1.3333333333333333, '..."
1,0.001518,0.002743,0.001518,0.000296,0.000000,0.000959,0.000000,0.003039,[],"[{'l_h': 8900.0, 'lh_h': 1.6952380952380952, '..."
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,[],"[{'l_h': 3400.0, 'lh_h': 0.6476190476190476, '..."
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,[],"[{'l_h': 1900.0000000000005, 'lh_h': 0.6666666..."
4,0.000767,0.000767,0.000384,0.000000,0.000000,0.000000,0.000000,0.000767,[],"[{'l_h': 3600.0, 'lh_h': 1.263157894736842, 'd..."
5,0.000348,0.000348,0.000000,0.000348,0.000245,0.001710,0.000092,0.000000,"[{'l_s': 5200.0, 'lh_s': 2.08, 'dl_s': 0.00051...","[{'l_h': 5600.000000000001, 'lh_h': 1.12, 'dl_..."
